# Stage 0. Library Import and Configuration Settings

In [1]:
import numpy as np
from config import cfg
from data import *
from models.cae import *
from trainer import *
from visualization import *
from utils import *

In [2]:
data_dir = 'C:/Users/kerry/OneDrive/Desktop/Projects/Datasets/MAD/head_ct'
result_dir = 'C:/Users/kerry/OneDrive/Desktop/Projects/UAD/results/HeadCT_patchCAE_128_64_16'

# Stage 1. Data Loading and Preprocessing

In [3]:
good_paths, bad_paths = load_paths(data_dir, cfg.good_label, cfg.bad_label)
print(f'Normal samples: {len(good_paths)}')
print(f'Abnormal samples: {len(bad_paths)}')

good_folds = kfold(paths=good_paths, k=cfg.k)
bad_folds = kfold(paths=bad_paths, k=2)

Normal samples: 100
Abnormal samples: 100


In [4]:
# dataloaders = build_dataloaders(good_folds, bad_folds, good_round=0, bad_round=0)
# images = next(iter(dataloaders['train']))[0]
# for i in range(images.shape[0]):
#     plt.imshow(images[i, 0], cmap='bone')
#     plt.show()

# Stage 2. Model Training, Validation, and Testing

In [5]:
set_random_seed(cfg.seed)
torch.multiprocessing.set_sharing_strategy('file_system')
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True

for good_round in range(5):
    for bad_round in range(2):
        train_cae(good_folds, bad_folds, good_round, bad_round, result_dir, patch=True)
        model = models.cae.AutoEncoder(patch=True).cuda()
        model.load_state_dict(torch.load(f'{result_dir}/best_models/Good{good_round}_Bad{bad_round}.pt'))
        model.eval()
        dataloaders = build_dataloaders(good_folds, bad_folds, good_round, bad_round, patch=True)
        print(validate_pcae(model, dataloaders['test'])[1])

0_0: 100%|███████| 1000/1000 [23:29<00:00,  1.41s/it, AUC=0.7510 (best=0.8300 epoch=60)]


0.704


0_1: 100%|███████| 1000/1000 [23:25<00:00,  1.41s/it, AUC=0.6480 (best=0.7420 epoch=60)]


0.788


1_0: 100%|███████| 1000/1000 [23:31<00:00,  1.41s/it, AUC=0.7320 (best=0.8020 epoch=30)]


...

0.821


4_0: 100%|███████| 1000/1000 [23:27<00:00,  1.41s/it, AUC=0.7640 (best=0.8320 epoch=90)]


0.732


4_1: 100%|███████| 1000/1000 [23:40<00:00,  1.42s/it, AUC=0.7370 (best=0.7690 epoch=40)]


0.761


In [ ]:
# BN        No bn
# 0.637 =>  0.672
# 0.69  =>  0.739
# 0.703
# 0.807

In [ ]:
# NoBN ??????
# _C.image_size = 64      128    128    128
# _C.patch_size = 32      32     64     64
# _C.stride =     8       16     8      16
# 0_0             0.672
# 1_0             0.739

In [6]:
np.mean([
0.704,
0.788,
0.731,
0.8220000000000001,
0.7739999999999999,
0.85,
0.78,
0.821,
0.732,
0.761])

0.7763